## Пример решения

Kaggle

In [1]:
import numpy
import pandas
import sklearn

import xgboost as xgb

Загружаем датасет

In [2]:
train = pandas.read_csv('Train.csv')
test = pandas.read_csv('Test.csv')

In [3]:
print(train.shape)

(100000, 25)


In [4]:
train.head()

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,0,2011-1,121,1.0,3,52,2,1,30.0,0.0,...,0,0,0,0,0,0,0,0,0,3090000
1,1,2011-1,58,1.0,6,101,3,1,5.0,NaN,...,0,0,0,0,0,0,0,0,0,15372000
2,2,2011-1,530,0.0,11,71,3,0,30.0,NaN,...,0,0,0,0,0,0,0,0,0,10067000
3,3,2011-1,448,NaN,5,36,1,1,15.0,0.0,...,0,0,0,0,0,0,0,0,0,3159000
4,4,2011-1,404,2.0,3,215,6,2,20.0,1.0,...,0,0,0,0,0,0,0,0,0,13816000


С помощью метода describe() получим некоторую сводную информацию по всей таблице. По умолчанию будет выдана информация только для количественных признаков. Это общее их количество (count), среднее значение (mean), стандартное отклонение (std), минимальное (min), макcимальное (max) значения, медиана (50%) и значения нижнего (25%) и верхнего (75%) квартилей:

In [5]:
train.describe()

,id,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
count,100000.000000,100000.000000,69896.000000,100000.000000,100000.000000,100000.00000,100000.000000,95047.000000,69943.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,1.000000e+05
mean,49999.500000,334.506830,0.539015,5.280490,52.108100,2.11278,0.402810,21.799846,0.497762,2.480110,...,0.004090,0.001460,0.000650,0.00801,0.008320,0.001800,0.000290,0.001420,0.000220,5.329125e+06
std,28867.657797,194.178711,0.578554,3.947259,17.433424,0.82522,0.550869,8.342282,0.499999,1.945282,...,0.063823,0.038182,0.025487,0.08914,0.090834,0.042388,0.017027,0.037656,0.014831,4.669177e+06
min,0.000000,0.000000,0.000000,1.000000,29.000000,1.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,3.940000e+05
25%,24999.750000,165.000000,0.000000,2.000000,40.000000,1.00000,0.000000,15.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2.673000e+06
50%,49999.500000,334.000000,0.000000,4.000000,52.000000,2.00000,0.000000,25.000000,0.000000,2.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,4.169500e+06
75%,74999.250000,503.000000,1.000000,7.000000,60.000000,3.00000,1.000000,30.000000,1.000000,4.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,6.414000e+06
max,99999.000000,671.000000,2.000000,25.000000,217.000000,6.00000,2.000000,30.000000,1.000000,11.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.104980e+08


Узнать количество заполненных (непропущенных) элементов можно с помощью метода count. Параметр axis = 0 указывает, что мы двигаемся по размерности 0 (сверху вниз), а не размерности 1 (слева направо), т.е. нас интересует количество заполненных элементов в каждом столбце, а не строке:

In [6]:
train.count(axis=0)

id            100000
date          100000
street_id     100000
build_tech     69896
floor         100000
area          100000
rooms         100000
balcon        100000
metro_dist     95047
g_lift         69943
n_photos      100000
kw1           100000
kw2           100000
kw3           100000
kw4           100000
kw5           100000
kw6           100000
kw7           100000
kw8           100000
kw9           100000
kw10          100000
kw11          100000
kw12          100000
kw13          100000
price         100000
dtype: int64

Заполнить пропущенные значения можно с помощью метода fillna. Заполним, например, медианными значениями.

axis=0 по-прежнему указывает, что мы двигаемся сверху вниз:

In [7]:
train = train.fillna(train.median(axis=0), axis=0)

In [8]:
train.count(axis=0)

id            100000
date          100000
street_id     100000
build_tech    100000
floor         100000
area          100000
rooms         100000
balcon        100000
metro_dist    100000
g_lift        100000
n_photos      100000
kw1           100000
kw2           100000
kw3           100000
kw4           100000
kw5           100000
kw6           100000
kw7           100000
kw8           100000
kw9           100000
kw10          100000
kw11          100000
kw12          100000
kw13          100000
price         100000
dtype: int64

In [9]:
train['date'].describe()

count     100000
unique        18
top       2011-8
freq        5735
Name: date, dtype: object

In [10]:
categorical_columns = [c for c in train.columns if train[c].dtype.name == 'object']
numerical_columns   = [c for c in train.columns if train[c].dtype.name != 'object']
print(categorical_columns)
print(numerical_columns)

['date']
['id', 'street_id', 'build_tech', 'floor', 'area', 'rooms', 'balcon', 'metro_dist', 'g_lift', 'n_photos', 'kw1', 'kw2', 'kw3', 'kw4', 'kw5', 'kw6', 'kw7', 'kw8', 'kw9', 'kw10', 'kw11', 'kw12', 'kw13', 'price']


пока что уберем колонку дата из выборки

Подготавляваем признаки

In [11]:
y_train = train['price'].values

In [12]:
X_train = train[['street_id',
          'build_tech',
          'floor',
          'area',
          'rooms',
          'balcon',
          'metro_dist',
          'g_lift',
          'n_photos',
          'kw1',
          'kw2',
          'kw3',
          'kw4',
          'kw5',
          'kw6',
          'kw7',
          'kw8',
          'kw9',
          'kw10',
          'kw11',
          'kw12',
          'kw13']].values

In [13]:
X_test = test[['street_id',
          'build_tech',
          'floor',
          'area',
          'rooms',
          'balcon',
          'metro_dist',
          'g_lift',
          'n_photos',
          'kw1',
          'kw2',
          'kw3',
          'kw4',
          'kw5',
          'kw6',
          'kw7',
          'kw8',
          'kw9',
          'kw10',
          'kw11',
          'kw12',
          'kw13']].values

Выполняем преобразование признаков

In [14]:
sklearn.preprocessing.MinMaxScaler().fit_transform(X_train, y_train)

array([[0.18032787, 0.5       , 0.08333333, ..., 0.        , 0.        ,
        0.        ],
       [0.08643815, 0.5       , 0.20833333, ..., 0.        , 0.        ,
        0.        ],
       [0.78986587, 0.        , 0.41666667, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.42622951, 0.        , 0.25      , ..., 0.        , 0.        ,
        0.        ],
       [0.61400894, 0.        , 0.16666667, ..., 0.        , 0.        ,
        0.        ],
       [0.36661699, 0.        , 0.08333333, ..., 0.        , 0.        ,
        0.        ]])

Обучаться, или, как говорят, строить модель, мы будем на обучающей выборке, а проверять качество построенной модели – на тестовой. Разбиение на тестовую и обучающую выборку должно быть случайным. Обычно используют разбиения в пропорции 50%:50%, 60%:40%, 75%:25% и т.д.

Мы воспользуемся функцией train_test_split из модуля sklearn.cross_validation. и разобьем данные на обучающую/тестовую выборки в отношении 70%:30%:

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print(N_train, N_test)

100000 100000


GBT – градиентный бустинг деревьев решений¶
GBT – еще один метод, строящий ансамбль деревьев решений. На каждой итерации строится новый классификатор, аппроксимирующий значение градиента функции потерь.

In [18]:
from sklearn import ensemble

In [ ]:
gbt = ensemble.GradientBoostingClassifier(n_estimators=50, verbose=2)
gbt.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 


In [ ]:
err_train = np.mean(y_train != gbt.predict(X_train))
print(err_train)

In [ ]:
print(gbt.predict(X_test))